In [89]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import scipy

In [90]:
data = pd.read_csv("data.csv")
data = data.rename(columns={"1990":pd.to_datetime("1990"),
                            "1991":pd.to_datetime("1991"),
                            "1992":pd.to_datetime("1992"),
                            "1993":pd.to_datetime("1993"),
                            "1994":pd.to_datetime("1994"),
                            "1995":pd.to_datetime("1995"),
                            "1996":pd.to_datetime("1996"),
                            "1997":pd.to_datetime("1997"),
                            "2005":pd.to_datetime("2005"),
                            "2006":pd.to_datetime("2006"),
                            "2007":pd.to_datetime("2007"),
                            "2008":pd.to_datetime("2008"),})
for i in ["1998","1999","2000","2001","2002","2003","2004",]:
    data[pd.to_datetime(i)] = "-" 
data = data.replace("-",np.nan)

In [120]:
df = pd.DataFrame()
for index, row in data.iterrows():
    try:
        df = df.append(row[2:].astype(float).interpolate(method="polynomial", order=2),ignore_index=True)
    except Exception as e:
        print(e)

The number of derivatives at boundaries does not match: expected 1, got 0+0


In [125]:
df["Crop"] = data["Crop"]
df["State"] = data["State"]
data = df

In [126]:
paddy = data.loc[data['Crop']=='Paddy']
jowar = data.loc[data['Crop']=='Jowar']
bajra = data.loc[data['Crop']=='Bajra']
maize = data.loc[data['Crop']=='Maize']
ragi = data.loc[data['Crop']=='Ragi']
wheat = data.loc[data['Crop']=='Wheat']
barley = data.loc[data['Crop']=='Barley']
gram = data.loc[data['Crop']=='Gram']
urad = data.loc[data['Crop']=='Urad']
moong = data.loc[data['Crop']=='Moong']
arhar = data.loc[data['Crop']=='Arhar']
sunflower = data.loc[data['Crop']=='Sunflower']
groundnut = data.loc[data['Crop']=='Groundnut']
rapeseed_mustard = data.loc[data['Crop']=='Rapeseed & Mustard']
safflower = data.loc[data['Crop']=='Safflower']
soyabean = data.loc[data['Crop']=='Soyabean']
cotton = data.loc[data['Crop']=='Cotton']
onion = data.loc[data['Crop']=='Onion']
sugarcane = data.loc[data['Crop']=='Sugercane']


In [127]:
sugarcane

,1990-01-01 00:00:00,1991-01-01 00:00:00,1992-01-01 00:00:00,1993-01-01 00:00:00,1994-01-01 00:00:00,1995-01-01 00:00:00,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,...,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,Crop,State
51,27.23,22.942000,25.36,34.518198,43.720000,47.420000,51.912325,58.265879,66.445943,76.467353,...,98.898629,104.939518,106.487800,103.543476,96.08,89.0,106.06,119.72,Sugercane,Andhra Pradesh
52,23.03,27.452753,31.80,36.083340,40.360787,45.130000,50.471512,56.080000,61.602509,67.054458,...,77.761130,82.512392,86.215464,88.870346,90.48,92.0,96.92,92.39,Sugercane,Bihar
53,NaN,NaN,NaN,NaN,35.200000,35.212516,36.900000,40.273963,45.315970,52.032944,...,69.855352,76.293072,79.104279,78.288972,73.83,64.0,48.46,86.53,Sugercane,Haryana
54,21.12,23.759831,27.37,31.964384,35.690000,37.020000,38.310000,43.590000,51.525959,61.798196,...,89.395656,101.832088,106.902720,104.607553,94.91,80.0,76.29,107.56,Sugercane,Karnataka
55,NaN,NaN,NaN,NaN,27.950000,34.547092,40.721715,46.489049,51.817578,56.723639,...,65.278902,68.916401,72.131431,74.923993,77.30,78.0,73.45,85.79,Sugercane,Maharashtra
56,22.34,25.090000,25.07,34.000000,41.950000,41.970000,43.510000,43.080000,43.765434,46.247491,...,56.620587,62.619601,66.659567,68.740486,68.86,68.0,73.35,93.64,Sugercane,Tamil Nadu


In [128]:
single_line = sugarcane.loc[sugarcane["State"]=="Andhra Pradesh"]
single_line

,1990-01-01 00:00:00,1991-01-01 00:00:00,1992-01-01 00:00:00,1993-01-01 00:00:00,1994-01-01 00:00:00,1995-01-01 00:00:00,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,...,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,Crop,State
51,27.23,22.942,25.36,34.518198,43.72,47.42,51.912325,58.265879,66.445943,76.467353,...,98.898629,104.939518,106.4878,103.543476,96.08,89.0,106.06,119.72,Sugercane,Andhra Pradesh


In [129]:
single_line

,1990-01-01 00:00:00,1991-01-01 00:00:00,1992-01-01 00:00:00,1993-01-01 00:00:00,1994-01-01 00:00:00,1995-01-01 00:00:00,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,...,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,Crop,State
51,27.23,22.942,25.36,34.518198,43.72,47.42,51.912325,58.265879,66.445943,76.467353,...,98.898629,104.939518,106.4878,103.543476,96.08,89.0,106.06,119.72,Sugercane,Andhra Pradesh
